<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/LaBSEAligner_SelfVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
!pip install scipy
!pip install sentence-splitter
!pip install indic-nlp-library

In [5]:
import sys
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import sentence_tokenize 
from sentence_transformers import SentenceTransformer
import numpy as np
import re 
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from scipy.spatial import distance

In [6]:
model = SentenceTransformer('LaBSE')

100%|██████████| 1.75G/1.75G [01:11<00:00, 24.4MB/s]


In [87]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [106]:
with open('/content/drive/MyDrive/eng_2.txt','r') as file:
  content = file.read()
patterns= ['-+','%',' \* ','\d{1}\.\d{1}','#','\d{1}','(\d{2}#)', '(\d{3}#)', '(\d{2}\.)', '(\d{3}\.)']
for pattern in patterns:
	content = re.sub(pattern,'',content)
content = content.replace('\n',' ')
splitter = SentenceSplitter(language='en')
so_sentences = splitter.split(content)
print('English Sentence Count is - '+ str(len(so_sentences)))


with open('/content/drive/MyDrive/tam_0_1.txt','r') as file:
  content = file.read()
patterns = ['!','}','{','\*','~','@','/','!  ','=','>>','“;','—',',','&',' \| ','\(\)','\)','\(  \)','\(','-+','%','[A-Z]+','[a-z]+',' \* ','#','\d{1}','(\d{2}\.)','(\d{4}\.)','(\d{3}\.)','(\d{1}\.)','(\d{2}#)', '(\d{3}#)','\d{4}','\d{3}','\d{2}','\d{4,}','\[','\]','\[\]' ] #CHANGE THIS IF THE LANGUAGE USES . as delimeter - For languages which have Poorna Virama as delimet
for pattern in patterns:
	content = re.sub(pattern,' ',content)
content = content.replace('\n', '  ')
content = content.replace('|','।')
content = re.sub('( \.)',' ',content)
content = re.sub('( . . .)+',' ',content)

ta_sentences = sentence_tokenize.sentence_split(content, lang='ta') 
print('Tamil Sentence Count is - '+ str(len(ta_sentences)))

English Sentence Count is - 716
Tamil Sentence Count is - 743


In [112]:
def LaBSEembeddings(source, target):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(source,show_progress_bar=True)
  embeddings_input_2 = model.encode(target,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  

In [113]:
def match_target_sentence(source_embedding, target_embeddings):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  # print("Match score: {}".format(min_distance))
  if min_distance >= 0.5:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     
      


In [115]:
def main():
  source_embeddings, target_embeddings = LaBSEembeddings(so_sentences, ta_sentences)
  target_map = {}
  source_map = {}
  for s in range(len(source_embeddings)):
    source_map[s] = so_sentences[s]
  for t in range(len(target_embeddings)):
    target_map[t] = ta_sentences[t]
  aligned_phrases= {}
  for s in range(len(source_embeddings)) :
      alignments = match_target_sentence([source_embeddings[s]], target_embeddings)
      if alignments is not None and alignments[2] is "MATCH":
          aligned_phrases[source_map[s]] =  target_map[alignments[0]]
  return aligned_phrases   


In [ ]:
aligned_phrases = main()

In [117]:
print(len(aligned_phrases))

676


In [118]:
ap = []
for key in aligned_phrases:
  l = []
  l.append(key)
  l.append(aligned_phrases[key])
  ap.append(l)

In [119]:
import csv
with open('matched.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['English', 'Tamil'])
    for a in ap:
      writer.writerow(a)